# Analisi Forense delle Immagini

Questo notebook analizza `coin.jpg` e `varroe.jpg` per trovare differenze strutturali e di compressione, anche quando le Tabelle di Quantizzazione sembrano identiche.

In [1]:
import PIL.Image
import PIL.ImageStat
from PIL.ExifTags import TAGS
import pandas as pd
import os
import numpy as np
from IPython.display import display
import skimage.measure

# Disabilita il limite per immagini enormi
PIL.Image.MAX_IMAGE_PIXELS = None

def get_exif_data(image):
    exif_data = {}
    raw_exif = image.getexif()
    if not raw_exif:
        return exif_data
    for tag_id, value in raw_exif.items():
        tag = TAGS.get(tag_id, tag_id)
        if isinstance(value, bytes):
            try: value = value.decode()
            except: value = str(value)
        exif_data[tag] = value
    return exif_data

def get_compression_details(image):
    info = {}
    # Subsampling (0 = 4:4:4, 1 = 4:2:2, 2 = 4:2:0)
    info['Subsampling'] = image.info.get('subsampling', 'Unknown')
    
    # Huffman Tables (se disponibili)
    huffman = image.info.get('huffman', 'Not optimized')
    info['Huffman Optimized'] = "Yes" if huffman != 'Not optimized' else "No"
    
    return info

def analyze_image(filepath):
    if not os.path.exists(filepath):
        return None
    
    file_size = os.path.getsize(filepath)
    
    with PIL.Image.open(filepath) as img:
        stats = PIL.ImageStat.Stat(img)
        
        info = {
            'Filename': os.path.basename(filepath),
            'Filesize (MB)': round(file_size / (1024*1024), 2),
            'Dimensions': img.size,
            'MP': round((img.size[0] * img.size[1]) / 1_000_000, 1),
            'Mean Brightness': round(np.mean(stats.mean), 2),
            'Std Dev (Noise)': round(np.mean(stats.stddev), 2),
            'Entropy': round(img.entropy(), 4)
        }
        
        # Aggiungi info compressione
        info.update(get_compression_details(img))
        
        # EXIF
        exif = get_exif_data(img)
        info['Date/Time'] = exif.get('DateTime', 'N/A')
        info['ISO'] = exif.get('ISOSpeedRatings', 'N/A')
        
        return info

files = ['coin.jpg', 'varroe.jpg']
results = [analyze_image(f) for f in files]
results = [r for r in results if r]

df = pd.DataFrame(results).T
if not df.empty:
    df.columns = [r['Filename'] for r in results]
    
    # Split into two tables: Main Analysis and Raw EXIF
    main_rows = ['Filename', 'Filesize (MB)', 'Dimensions', 'MP', 'Mean Brightness', 'Std Dev (Noise)', 'Entropy', 'Subsampling', 'Huffman Optimized']
    # Filter main rows that actually exist in the dataframe
    main_rows = [r for r in main_rows if r in df.index]
    
    exif_rows = [idx for idx in df.index if idx not in main_rows]
    
    print("--- Analisi Strutturale & Qualita ---")
    display(df.loc[main_rows])
    
    print("\n--- Dati EXIF Completi ---")
    display(df.loc[exif_rows].sort_index())
else:
    print("Immagini non trovate.")


--- Analisi Strutturale & Qualita ---


,coin.jpg,varroe.jpg
Filename,coin.jpg,varroe.jpg
Filesize (MB),18.68,25.79
Dimensions,"(9024, 12032)","(12032, 9024)"
MP,108.6,108.6
Mean Brightness,157.12,163.76
Std Dev (Noise),28.08,45.79
Entropy,8.1064,8.4458
Subsampling,Unknown,Unknown
Huffman Optimized,No,No



--- Dati EXIF Completi ---


,coin.jpg,varroe.jpg
Date/Time,2026:01:02 18:23:50,2026:01:02 18:39:52
ISO,N/A,N/A
